# Chapter 6: Advanced Spark Programming (Python)

In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Advanced-Spark-Programming").master("local[*]").getOrCreate()
sc = spark.sparkContext

## Accumulators

In [35]:
acc_5 = sc.accumulator(0)

In [36]:
num_rdd = sc.parallelize([1,2,5,5,3,6,9,6,5,9])

In [37]:
num_rdd.glom().collect()

[[1, 2], [5, 5], [3, 6], [9, 6, 5, 9]]

In [38]:
def count_5(num):
    global acc_5
    if num == 5:
        acc_5 += 1

In [39]:
num_rdd.map(count_5).collect()

[None, None, None, None, None, None, None, None, None, None]

In [40]:
acc_5

Accumulator<id=4, value=3>